In [ ]:
#@title **Choose song** { run: "auto" }
 
Search = "\u05E2\u05D5\u05D3 \u05D7\u05D5\u05D6\u05E8" #@param {type:"string"}
Sort_by = "Artist" #@param ['Song','Artist']
Progression_value = "0" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)
 
# Load modules
from IPython.display import Javascript,clear_output,display
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
from google.colab import output
import uuid
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,jslink
import ipywidgets as widgets
import copy
from pprint import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import re
import IPython
from urllib.request import Request, urlopen
import pandas as pd
import requests
from io import StringIO
try: first_run # A way to check if element exist (i wanted to run the imports only on th first time)
except NameError: first_run = True
 
if first_run==True:
  !pip install pychord -q
  #from __future__ import print_function
  first_run = False
  # Constants
  text_alignment = 'right'
  # import song list
  #------------------------------------------------------------------------------------
  orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
  file_id = orig_url.split('/')[-2]
  dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
  text = requests.get(dwn_url)
  text.encoding = 'utf-8'
  csv_raw = StringIO(text.text)
  df = pd.read_csv(csv_raw,encoding='utf-8')
  df['אמן'] = df['אמן'].apply(lambda x: x.strip())
  df['שיר'] = df['שיר'].apply(lambda x: x.strip())
 
from pychord import Chord
# Chord_Progression function
def Chord_Progression(orig_chord=str,transpose_level=0):
  prog_chord = Chord(orig_chord)
  prog_chord.transpose(transpose_level)
  return prog_chord.chord
 
# Choose song list from spesific artist to display
def artist_songs(artist_name=str):
  song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()
  song_list = list(zip(song_list,song_list.index))
  return song_list
#------------------------------------------------------------------------------------
# Choose roder of view in the select table
if Sort_by == "Song":
  df['shir_aman'] = df['שיר']+" | "+df['אמן']
else:
  df['shir_aman'] = df['אמן']+" | "+df['שיר'] 
song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='40%', height='200px',right='True'),rows=20)
display(song_tab)   #!!!
 
class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
 
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
 
    template = """<style>
    #{callback_id} {{
      background-color: #71A7FF;
      border: none;
      color: white;
      padding: 10px 30px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 10px 10px;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #CFDFF9;
      font-size : 20px;
    }}
 
    #{callback_id}:hover {{background-color: #5797FE}}
    #{callback_id}:active {{
      background-color: #71A7FF;
      box-shadow: 0 4px #CFDFF9;
      transform: translateY(4px);
      }}
 
    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
 
class InvokeButton1(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
 
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
 
    template = """<style>
    #{callback_id} {{
      background-color: #F688FF;
      border: none;
      color: white;
      padding: 13px 42px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: 10px 10px;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #F7E0F9;
      font-size : 20px;
    }}
 
    #{callback_id}:hover {{background-color: #EF38FE}}
    #{callback_id}:active {{
      background-color: #F688FF;
      box-shadow: 0 4px #F7E0F9;
      transform: translateY(4px);
      }}
 
    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
 
def on_button_clicked(b):
  #clear_output()
  #with output:
  #display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
  #clear_output()
  myurl = df.iloc[song_tab.value]['לינק']
  req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
  song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
  header = song.find(attrs={"class":"song_block_title"})
  # Clean the HTML from ads and more unnecessary stuff
  header = header.get_text().strip().replace("אקורדים לשיר ","")
  text_alignment = 'right'
  header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
  on_button_clicked.header = header
 
  progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
  header_progression = header+progression_status
 
  delete_nodes = [{"class":"song_rating"},
                  {"class":"row"},
                  {"id":"songad"},
                  {"class":"song_block_title"},
                  {"id":"songTopAd8Td"},
                  {"class":"videosTitle"},
                  {"type":"text/javascript"},
                  {"id":"taboola-below-article-thumbnails"},
                  {"id":"downInSongTable"},
                  {"class":"chord_info"}]
                  
  for node in delete_nodes:
    for div in song.find_all(attrs=node): 
        div.decompose()
 
  # Align guitar TABS to the left
  tab_list = ''
  tab_nodes = [{"class":"tabs"}]
  for node in tab_nodes:
    for div in song.find_all(attrs=node):
      div['dir'] = 'ltr'
      div['align'] = 'left'
      #div['display'] = 'inline-block'
      #div['width'] = '300px'
      div['position'] = 'absolute'
      div['right'] = '10%'
      #div['margin-left'] = '300px'
      tab_list=tab_list+'<p'+str(div)+'</p>'
  on_button_clicked.song=song
  # Merge Header & Song(words+chords)
  full_page=header_progression+str(song)
  on_button_clicked.full_page = full_page
 #display(IPython.display.HTML(full_page))
 
  return on_button_clicked.song,on_button_clicked.header,on_button_clicked.full_page
 
def modulate_full_page(Progression_song,header):
  for elm in Progression_song.find_all('span'):
    try:
      elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
    except:
      dummy = 0
  if Prog_value > 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  elif Prog_value < 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  else:
    progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
    header_progression = header+progression_status
  # Merge Header & Song(words+chords)
  full_page=header_progression+str(Progression_song)
  return full_page
dont_need_button=False
try:
  Progression_song = copy.copy(download_song.song)
  header = copy.copy(download_song.header)
  display(InvokeButton('🤩 Get my Song 🎤', download_song),InvokeButton1('  Transform Chords  ', modulazion))
  dont_need_button = True
except:
  with output.use_tags('float_line'):
    display(IPython.display.HTML("<marquee style='width: 50%; color: purple;font-size : 20px;'><b>✌ נא לבחור שיר</b></marquee>"))
 
 
def modulazion():
  #Progression_song = copy.copy(download_song.header)
  #header = copy.copy(download_song.header)
  full_page = modulate_full_page(Progression_song=Progression_song,header=header)
  output.clear(output_tags='download_song_output')
  output.clear(output_tags='modulazion_output')
  with output.use_tags('modulazion_output'):
    #print(sing)
    display(IPython.display.HTML(full_page))
 
def download_song():
  output.clear(output_tags='float_line')
  output.clear(output_tags='download_song_output')
  output.clear(output_tags='modulazion_output')
  on_button_clicked(b=True)
  with output.use_tags('download_song_output'):
    display(IPython.display.HTML(on_button_clicked.full_page))
    download_song.song=on_button_clicked.song
    download_song.header=on_button_clicked.header
    return download_song.song,download_song.header
 
 
 
 
  #display(song_tab,button,IPython.display.HTML(full_page)
if dont_need_button == False:
  display(InvokeButton('🤩 Get my Song 🎤', download_song),InvokeButton1('  Transform Chords  ', modulazion))

<IPython.core.display.Javascript object>

Select(layout=Layout(height='200px', right='True', width='40%'), options=(('ברי סחרוף | עוד חוזר הניגון', 545)…

In [ ]:
del(first_run)



In [ ]:
class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<style>
    #{callback_id} {{
      background-color: #71A7FF;
      border: none;
      color: white;
      padding: 1%;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: auto;
      width: 50%;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #CFDFF9;
      font-size : 20px;
    }}

    #{callback_id}:hover {{background-color: #5797FE}}
    #{callback_id}:active {{
      background-color: #71A7FF;
      box-shadow: 0 4px #CFDFF9;
      transform: translateY(4px);
      }}

    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

class InvokeButton1(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<style>
    #{callback_id} {{
      background-color: #F688FF;
      border: none;
      color: white;
      padding: 1.3%;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      margin: auto;
      width: 50%;
      cursor: pointer;
      border-radius: 16px;
      box-shadow: 0 9px #F7E0F9;
      font-size : 20px;
    }}

    #{callback_id}:hover {{background-color: #EF38FE}}
    #{callback_id}:active {{
      background-color: #F688FF;
      box-shadow: 0 4px #F7E0F9;
      transform: translateY(4px);
      }}

    </style>
    <button id="{callback_id}">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

In [ ]:
from ipywidgets import Button, Layout, jslink, IntText, IntSlider

Invoke_song_button = InvokeButton("🤩 Get my Song 🎤", download_song)._repr_html_()
HTML_song_button = widgets.HTML(Invoke_song_button)

Invoke_chords_button = InvokeButton1('  Transform Chords  ', modulazion)._repr_html_()
HTML_chords_button = widgets.HTML(Invoke_chords_button)
HTML_song_button

HTML(value='<style>\n    #button-307c25c2-52c8-4768-8f40-fc828657c538 {\n      background-color: #71A7FF;\n   …

In [ ]:
# Utils widgets
from ipywidgets import Button, Layout, jslink, IntText, IntSlider

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

#song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='40%', height='200px',right='True'),rows=20)
song_tab = widgets.Select(options=artist_songs(Search),layout=Layout(width='50%', height='auto',right = 'True'))


top_right_text = widgets.Dropdown(
                  options=[6,5,4,3,2,1,0,-1,-2,-3,-4,-5],
                  value=0,
                  layout=Layout(width='50%', height='60%',align_self  = 'center'),
                  disabled=False)
#top_right_text = IntText(description='Top right', layout=Layout(width='auto', height='auto'))

from ipywidgets import TwoByTwoLayout

TwoByTwoLayout(top_left=song_tab,
               bottom_left=HTML_song_button,
               top_right=top_right_text,
               bottom_right=HTML_chords_button)


TwoByTwoLayout(children=(Select(layout=Layout(grid_area='top-left', height='auto', right='True', width='50%'),…

""
פתיחה:
Am G Am G Am G Am G
e|-7--5----------------------------|
B|---------8--7----5---------------|
G|----------------------7---6------|
D|------------------------------7--|
A|---------------------------------|
E|---------------------------------|
Am G Am G
e|-10----------------------------------|


In [ ]:
widgets.SelectionSlider(
    options=['scrambled', 'sunny side up', 'poached', 'over easy'],
    value='sunny side up',
    description='I like my eggs ...',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

import datetime
dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='Months (2015)',
    disabled=False
)


ToggleButtons(description='Speed:', options=('Slow', 'Regular', 'Fast'), tooltips=('Description of slow', 'Des…

# Production

In [ ]:
#@title **Choose song** { run: "auto" }
from IPython.display import Javascript,clear_output
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
# Load modules
#------------------------------------------------------------------------------------
try: first_run # A way to check if element exist (i wanted to run the imports only on th first time)
except NameError: first_run = True

if first_run==True:
  !pip install pychord -q
  from pychord import Chord
  import copy
  from pprint import pprint
  from bs4 import BeautifulSoup
  from urllib.request import urlopen 
  import re
  import IPython
  from IPython.display import clear_output
  from google.colab import output
  from urllib.request import Request, urlopen
  import pandas as pd
  import requests
  from io import StringIO
  #from __future__ import print_function
  from ipywidgets import interact, interactive, fixed, interact_manual,Layout,jslink
  import ipywidgets as widgets
  first_run = False

  # import song list
  #------------------------------------------------------------------------------------
  orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
  file_id = orig_url.split('/')[-2]
  dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
  text = requests.get(dwn_url)
  text.encoding = 'utf-8'
  csv_raw = StringIO(text.text)
  df = pd.read_csv(csv_raw,encoding='utf-8')
  df['אמן'] = df['אמן'].apply(lambda x: x.strip())
  df['שיר'] = df['שיר'].apply(lambda x: x.strip())


# choose song - output url 
#------------------------------------------------------------------------------------
Search = "\u05D1\u05E8\u05D9 \u05E1" #@param {type:"string"}
Sort_by = "Song" #@param ['Song','Artist']
if Sort_by == "Song":
  df['shir_aman'] = df['שיר']+" | "+df['אמן']
else:
  df['shir_aman'] = df['אמן']+" | "+df['שיר']

def artist_songs(artist_name=str):
  song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()

  song_list = list(zip(song_list,song_list.index))
  return song_list
  
song_tab = widgets.Select(options=artist_songs(Search),
                          description='Song:',
                          layout=Layout(width='40%', height='200px',right='True'),
                          rows=20)
display(song_tab)

# -----------------------------------------------------------------------------
#try: on_button_clicked.header
#except NameError: Dont_load_song = False
Progression_value = "2" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)
try:
  on_button_clicked.header
  print('Dont_load_song')
  Progression_song = copy.copy(on_button_clicked.song)
  header = copy.copy(on_button_clicked.header)
  # Enable chord progression!
  def Chord_Progression(orig_chord=str,transpose_level=0):
    """
    :param orig_chord type: str
    :param transpose_level: The level (in half tone) you want to transpose the chord. type: int, default 0
    :return prog_chord type: str
      usage:
        >>> Chord_Progression(orig_chord='Am7/G',transpose_level=6)
        output: "Ebm7/Db"
    """
    prog_chord = Chord(orig_chord)
    prog_chord.transpose(transpose_level)
    return prog_chord.chord

  for elm in Progression_song.find_all('span'):
      try:
        elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
      except:
        pass
  text_alignment = 'right'
  if Prog_value > 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  elif Prog_value < 0:
    progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
    header_progression = header+progression_status
  else:
    #header_progression = header
    progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
    header_progression = header+progression_status

  # Merge Header & Song(words+chords)
  clear_output(True)
  full_page=header_progression+str(Progression_song)
  new_page = IPython.display.HTML(full_page)
  display(song_tab,button,IPython.display.HTML(full_page))
  #Dont_load_song = True
except:
  #display(song_tab,button,IPython.display.HTML(full_page))
  print('Yes_load_song')
  
#-----------------------------------------------------------------------------
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="🤩 Get my Song 🎤",
                        style = {'button_color': 'lightblue'}, 
                        layout = Layout(width = '160px'))
button.style.font_weight='900'
output = widgets.Output()
song_just_choosen = False 
def on_button_clicked(b):
  # Display the message within the output widget.
  #Prog_value = 0
  with output:
    Prog_value = 0
    display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
    clear_output(True)
    on_button_clicked.song_just_choosen = True 
    Prog_value = 0
    myurl = df.iloc[song_tab.value]['לינק']
    req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
    song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
    header = song.find(attrs={"class":"song_block_title"})
    # Clean the HTML from ads and more unnecessary stuff
    header = header.get_text().strip().replace("אקורדים לשיר ","")
    text_alignment = 'right'
    header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
    on_button_clicked.header = header
    if Prog_value > 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    elif Prog_value < 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    else:
      #header_progression = header
      progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
      header_progression = header+progression_status

    delete_nodes = [{"class":"song_rating"},
                    {"class":"row"},
                    {"id":"songad"},
                    {"class":"song_block_title"},
                    {"id":"songTopAd8Td"},
                    {"class":"videosTitle"},
                    {"type":"text/javascript"},
                    {"id":"taboola-below-article-thumbnails"},
                    {"id":"downInSongTable"},
                    {"class":"chord_info"}]
    for node in delete_nodes:
      for div in song.find_all(attrs=node): 
          div.decompose()

    # Align guitar TABS to the left
    tab_list = ''
    tab_nodes = [{"class":"tabs"}]
    for node in tab_nodes:
      for div in song.find_all(attrs=node):
        div['dir'] = 'ltr'
        div['align'] = 'left'
        #div['display'] = 'inline-block'
        #div['width'] = '300px'
        div['position'] = 'absolute'
        div['right'] = '10%'
        #div['margin-left'] = '300px'
        tab_list=tab_list+'<p'+str(div)+'</p>'
    on_button_clicked.song=song
    # Merge Header & Song(words+chords)
    full_page=header_progression+str(song)
    display(IPython.display.HTML(full_page))

    return on_button_clicked.song,on_button_clicked.header


button.on_click(on_button_clicked)

display(button, output)


<IPython.core.display.Javascript object>

Select(description='Song:', layout=Layout(height='200px', right='True', width='40%'), options=(('1900 | ברי סח…

Yes_load_song


Button(description='\U0001f929 Get my Song 🎤', layout=Layout(width='160px'), style=ButtonStyle(button_color='l…

Output()

רק בעם הראשונה

In [ ]:
Progression_value = "0" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)

if first_run==False:
  try: 
    on_button_clicked.header
    Progression_song = copy.copy(on_button_clicked.song)
    header = copy.copy(on_button_clicked.header)
    # Enable chord progression!
    def Chord_Progression(orig_chord=str,transpose_level=0):
      """
      :param orig_chord type: str
      :param transpose_level: The level (in half tone) you want to transpose the chord. type: int, default 0
      :return prog_chord type: str
        usage:
          >>> Chord_Progression(orig_chord='Am7/G',transpose_level=6)
          output: "Ebm7/Db"
      """
      prog_chord = Chord(orig_chord)
      prog_chord.transpose(transpose_level)
      return prog_chord.chord

    for elm in Progression_song.find_all('span'):
        try:
          elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
        except:
          pass
    text_alignment = 'right'
    if Prog_value > 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    elif Prog_value < 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    else:
      #header_progression = header
      progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
      header_progression = header+progression_status

    # Merge Header & Song(words+chords)
    clear_output(True)
    full_page=header_progression+str(Progression_song)
    display(song_tab,button,IPython.display.HTML(full_page))
  except:
    clear_output(True)
    display(song_tab,button,IPython.display.HTML(full_page))
    pass

Button(description='\U0001f929 Get my Song 🎤', layout=Layout(width='160px'), style=ButtonStyle(button_color='l…

Output()

In [ ]:
display(button)

Button(description='\U0001f929 Get my Song 🎤', layout=Layout(width='160px'), style=ButtonStyle(button_color='l…

In [ ]:
#display(button, output)

Progression_value = "-2" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)

if first_run==False:
  try: 
    on_button_clicked.header
    Progression_song = copy.copy(on_button_clicked.song)
    header = copy.copy(on_button_clicked.header)
    # Enable chord progression!
    def Chord_Progression(orig_chord=str,transpose_level=0):
      """
      :param orig_chord type: str
      :param transpose_level: The level (in half tone) you want to transpose the chord. type: int, default 0
      :return prog_chord type: str
        usage:
          >>> Chord_Progression(orig_chord='Am7/G',transpose_level=6)
          output: "Ebm7/Db"
      """
      prog_chord = Chord(orig_chord)
      prog_chord.transpose(transpose_level)
      return prog_chord.chord

    for elm in Progression_song.find_all('span'):
        try:
          elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
        except:
          pass
    text_alignment = 'right'
    if Prog_value > 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    elif Prog_value < 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    else:
      #header_progression = header
      progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
      header_progression = header+progression_status

    # Merge Header & Song(words+chords)
    full_page=header_progression+str(Progression_song)
    display(button,IPython.display.HTML(full_page))
  except: pass

In [ ]:
##@title **Choose song** { run: "auto" }

Progression_value = "-2" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 
Prog_value = int(Progression_value)
Progression_song = copy.copy(on_button_clicked.song)
header = copy.copy(on_button_clicked.header)
# Enable chord progression!
def Chord_Progression(orig_chord=str,transpose_level=0):
  """
  :param orig_chord type: str
  :param transpose_level: The level (in half tone) you want to transpose the chord. type: int, default 0
  :return prog_chord type: str
    usage:
      >>> Chord_Progression(orig_chord='Am7/G',transpose_level=6)
      output: "Ebm7/Db"
  """
  prog_chord = Chord(orig_chord)
  prog_chord.transpose(transpose_level)
  return prog_chord.chord

for elm in Progression_song.find_all('span'):
    try:
      elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
    except:
      pass
text_alignment = 'right'
if Prog_value > 0:
  progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
  header_progression = header+progression_status
elif Prog_value < 0:
  progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
  header_progression = header+progression_status
else:
  #header_progression = header
  progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
  header_progression = header+progression_status

# Merge Header & Song(words+chords)
full_page=header_progression+str(Progression_song)

display(IPython.display.HTML(full_page))

In [ ]:
import copy

Progression_song = copy.copy(on_button_clicked.song)
header = copy.copy(on_button_clicked.header)

reset = '0'

In [ ]:
int(Progression_value)

0

In [ ]:
# import song list
# choose song - output url
# scrap song
# show song
# transpose
on_button_clicked

In [ ]:
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="🤩 Get my Song 🎤",
                        style = {'button_color': 'lightblue'}, 
                        layout = Layout(width = '160px'))
button.style.font_weight='900'
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  #Prog_value = 0
  with output:

    Prog_value = 0
    myurl = df.iloc[song_tab.value]['לינק']
    req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
    song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
    header = song.find(attrs={"class":"song_block_title"})

    # Clean the HTML from ads and more unnecessary stuff
    header = header.get_text().strip().replace("אקורדים לשיר ","")
    text_alignment = 'right'
    header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'

    if Prog_value > 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    elif Prog_value < 0:
      progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
      header_progression = header+progression_status
    else:
      #header_progression = header
      progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
      header_progression = header+progression_status

    delete_nodes = [{"class":"song_rating"},
                    {"class":"row"},
                    {"id":"songad"},
                    {"class":"song_block_title"},
                    {"id":"songTopAd8Td"},
                    {"class":"videosTitle"},
                    {"type":"text/javascript"},
                    {"id":"taboola-below-article-thumbnails"},
                    {"id":"downInSongTable"},
                    {"class":"chord_info"}]
    for node in delete_nodes:
      for div in song.find_all(attrs=node): 
          div.decompose()

    # Align guitar TABS to the left
    tab_list = ''
    tab_nodes = [{"class":"tabs"}]
    for node in tab_nodes:
      for div in song.find_all(attrs=node):
        div['dir'] = 'ltr'
        div['align'] = 'left'
        #div['display'] = 'inline-block'
        #div['width'] = '300px'
        div['position'] = 'absolute'
        div['right'] = '10%'
        #div['margin-left'] = '300px'
        tab_list=tab_list+'<p'+str(div)+'</p>'

    # Merge Header & Song(words+chords)
    full_page=header_progression+str(song)

    display(IPython.display.HTML(full_page))

button.on_click(on_button_clicked)
display(button, output)

In [ ]:
Prog_value = 0
myurl = df.iloc[song_tab.value]['לינק']
req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
header = song.find(attrs={"class":"song_block_title"})

# Clean the HTML from ads and more unnecessary stuff
header = header.get_text().strip().replace("אקורדים לשיר ","")
text_alignment = 'right'
header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'

if Prog_value > 0:
  progression_status = f"""<h2 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h2>"
  header_progression = header+progression_status
elif Prog_value < 0:
  progression_status = f"""<h2 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h2>"
  header_progression = header+progression_status
else:
  #header_progression = header
  progression_status =f"""<h2 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h2>"
  header_progression = header+progression_status

delete_nodes = [{"class":"song_rating"},
                {"class":"row"},
                {"id":"songad"},
                {"class":"song_block_title"},
                {"id":"songTopAd8Td"},
                {"class":"videosTitle"},
                {"type":"text/javascript"},
                {"id":"taboola-below-article-thumbnails"},
                {"id":"downInSongTable"},
                {"class":"chord_info"}]
for node in delete_nodes:
  for div in song.find_all(attrs=node): 
      div.decompose()

# Align guitar TABS to the left
tab_list = ''
tab_nodes = [{"class":"tabs"}]
for node in tab_nodes:
  for div in song.find_all(attrs=node):
    div['dir'] = 'ltr'
    div['align'] = 'left'
    #div['display'] = 'inline-block'
    #div['width'] = '300px'
    div['position'] = 'absolute'
    div['right'] = '10%'
    #div['margin-left'] = '300px'
    tab_list=tab_list+'<p'+str(div)+'</p>'

# Merge Header & Song(words+chords)
full_page=header_progression+str(song)

display(IPython.display.HTML(full_page))

# Working code

### input url
### add run button

In [ ]:
#@title Setup
#@markdown Takes about 10 sec...
%%capture
!pip install pychord
from pychord import Chord
from pprint import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import re
import IPython
from google.colab import output
from urllib.request import Request, urlopen

In [ ]:
#@title **Copy the URL** to the desired song from www.tab4u.co.il { run: "auto" }
URL =  "https://www.tab4u.com/tabs/songs/1481_%D7%97%D7%9E%D7%99_%D7%A8%D7%95%D7%93%D7%A0%D7%A8_-_%D7%99%D7%94%D7%99%D7%95_%D7%A9%D7%99%D7%92%D7%99%D7%93%D7%95_%D7%A2%D7%9C%D7%99%D7%94.html" #@param {type:"string"}
Progression_value = "0" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,-5] 

myurl = URL
Prog_value = int(Progression_value)
req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
header = song.find(attrs={"class":"song_block_title"})

# Clean the HTML from ads and more unnecessary stuff
header = header.get_text().strip().replace("אקורדים לשיר ","")
text_alignment = 'right'
header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
#progression_status = ''
if Prog_value > 0:
  progression_status = f"""<h2 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h2>"
  header = header+progression_status
if Prog_value < 0:
  progression_status = f"""<h2 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h2>"
  header = header+progression_status
delete_nodes = [{"class":"song_rating"},
                {"class":"row"},
                {"id":"songad"},
                {"class":"song_block_title"},
                {"id":"songTopAd8Td"},
                {"class":"videosTitle"},
                {"type":"text/javascript"},
                {"id":"taboola-below-article-thumbnails"},
                {"id":"downInSongTable"},
                {"class":"chord_info"}]
for node in delete_nodes:
  for div in song.find_all(attrs=node): 
      div.decompose()

# Align guitar TABS to the left
tab_list = ''
tab_nodes = [{"class":"tabs"}]
for node in tab_nodes:
  for div in song.find_all(attrs=node):
    div['dir'] = 'ltr'
    div['align'] = 'left'
    #div['display'] = 'inline-block'
    #div['width'] = '300px'
    div['position'] = 'absolute'
    div['right'] = '10%'
    #div['margin-left'] = '300px'
    tab_list=tab_list+'<p'+str(div)+'</p>'

# Enable chord progression!
def Chord_Progression(orig_chord=str,transpose_level=0):
  """
  :param orig_chord type: str
  :param transpose_level: The level (in half tone) you want to transpose the chord. type: int, default 0
  :return prog_chord type: str
    usage:
      >>> Chord_Progression(orig_chord='Am7/G',transpose_level=6)
      output: "Ebm7/Db"
  """
  prog_chord = Chord(orig_chord)
  prog_chord.transpose(transpose_level)
  return prog_chord.chord

for elm in song.find_all('span'):
    try:
      elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
    except:
      pass
# Merge Header & Song(words+chords)
full_page=header+str(song)

display(IPython.display.HTML(full_page))

# Get song index

In [ ]:
from pprint import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import re
import IPython
from google.colab import output
from urllib.request import Request, urlopen

### Find total song number

In [ ]:
song_number_find_url = "https://www.tab4u.com/resultsSimple?tab=songs&type=song&q=*&content=chords&max_chords=0"
req = Request(song_number_find_url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
web_part = soupified.find("td", {"class": "foundTxtTd"})
song_number = [int(i) for i in web_part.get_text().split() if i.isdigit()][0]
song_number

### Scrape tables
https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [ ]:
#%%time
import numpy as np
import string
from tqdm import tqdm
from time import sleep

song_list =[]
#for first_index in np.arange(0,500,100):
for first_index in tqdm(np.arange(0,song_number,100)):
  index_url = f"https://www.tab4u.com/resultsSimple?tab=songs&q=&type=song&cat=&content=chords&max_chords=0&n=100&sort=&s={first_index}"
  req = Request(index_url, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()
  soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
  web_part = soupified.find_all("td", {"class": "songTd1"})
  
  for W in web_part:
    Song_name = W.find('a')['onmouseover'].split(sep=',')[-1].translate(str.maketrans('', '', string.punctuation))
    Artist_name = W.text
    Link2song = "https://www.tab4u.com/"+W.find('a')['href']
    song_list.append([Song_name,Artist_name,Link2song])

df=pd.DataFrame(song_list,columns = ["אמן","שיר","לינק"])
df.to_csv("/content/drive/MyDrive/Colab_yotam/music/SongList.csv",index=False,encoding='utf-8')
df

In [ ]:
# link to the public csv!!!!!
orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'

import pandas as pd
import requests
from io import StringIO

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
text = requests.get(dwn_url)
text.encoding = 'utf-8'
csv_raw = StringIO(text.text)
df = pd.read_csv(csv_raw,encoding='utf-8')
df['אמן'] = df['אמן'].apply(lambda x: x.strip())
df['שיר'] = df['שיר'].apply(lambda x: x.strip())
df.head()

,אמן,שיר,לינק
0,Christina Aguilera,Your Body,https://www.tab4u.com/tabs/songs/3_Christina_A...
1,Maroon 5,One More Night,https://www.tab4u.com/tabs/songs/5_Maroon_5_-_...
2,אבטיפוס,אשה אשה,https://www.tab4u.com/tabs/songs/20_%D7%90%D7%...
3,סינרגיה,צועקים על אהבה,https://www.tab4u.com/tabs/songs/26_%D7%A1%D7%...
4,אבטיפוס,מועקה,https://www.tab4u.com/tabs/songs/30_%D7%90%D7%...


# inteactive
https://dash.plotly.com/datatable

In [ ]:
from google.colab import data_table

data_table.DataTable(df,include_index=False, num_rows_per_page=10,max_rows=None, max_columns=3,min_width='10')


In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,jslink
import ipywidgets as widgets

In [ ]:
import ipywidgets as widgets
from widgets import Layout,jslink

In [ ]:
def f(x):
    return x
#interact(f, x=['apples','oranges']);
interact(f, x=widgets.Combobox(options=artist_list, value=""));

interactive(children=(Combobox(value='', description='x', options=('3 Doors Down', '4 Non Blondes', '5 Seconds…

In [ ]:
if s
df['shir_aman'] = df['אמן']+" | "+df['שיר']
df['shir_aman'] = df['שיר']+" | "+df['אמן']
df['shir_aman'] 

0          Your Body | Christina Aguilera
1               One More Night | Maroon 5
2                       אשה אשה | אבטיפוס
3                צועקים על אהבה | סינרגיה
4                         מועקה | אבטיפוס
                      ...                
8818                היינו שמחים | המדרגות
8819    למה את כבר לא עונה לי | מיכאל לוי
8820              יעלה יעלה | אנסמבל יעלה
8821                כמה רקדנו | בניה ברבי
8822                 יום הולדת | גקי שושן
Name: shir_aman, Length: 8823, dtype: object

In [ ]:
artist_list = df['אמן'].unique().tolist()
artist_list.sort()
#artist_list

In [ ]:
song_list = df['שיר'].sort_values()
song_list = list(zip(song_list,song_list.index))
#song_list

In [ ]:
#@title **Choose song** { run: "auto" }
Search = "\u05E2\u05D5\u05D3 \u05D7\u05D5" #@param {type:"string"}
Sort_by = "Artist" #@param ['Song','Artist']
if Sort_by == "Song":
  df['shir_aman'] = df['שיר']+" | "+df['אמן']
else:
  df['shir_aman'] = df['אמן']+" | "+df['שיר']

def artist_songs(artist_name=str):
  song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()

  song_list = list(zip(song_list,song_list.index))
  return song_list
  
song_tab = widgets.Select(options=artist_songs(Search),
                          description='Song:',
                          layout=Layout(width='40%', height='400px',right='True'),
                          rows=20)
display(song_tab)

Select(description='Song:', layout=Layout(height='400px', right='True', width='40%'), options=(('ברי סחרוף | ע…

In [ ]:
#@title new { run: "auto" }
Artist = "" #@param ['3 Doors Down', '4 Non Blondes', '5 Seconds of Summer', '8th Day', 'A1', 'ABBA', 'ACDC', 'ANNA RF', 'Acollective', 'Adam Ant', 'Adam Levine', 'Adele', 'Adi Ulmansky', 'Aerosmith', 'Afroman', 'Aha', 'Air', 'Air Supply', 'Airbag', 'Al Green', 'Aladdin', 'Alan Parsons Project', 'Alan Walker', 'Alanis Morissette', 'Alec Benjamin', 'Alekos Zazopoulos', 'Alex Turner', 'Alfred García', 'Alice In Chains', 'Alicia keys', 'All Saints', 'Aloe Blacc', 'Alphaville', 'America', 'Amy Macdonald', 'Amy Winehouse', 'Anouk', 'Arctic Monkeys', 'Aretha Franklin', 'Arif Ceylan', 'Aris San', 'Arrhwallaco', 'Asaf Avidan', 'Asaf Avidan  the Mojos', 'Augustana', 'Avenged Sevenfold', 'Avicii', 'Avril Lavigne', 'Awolnation', 'Backstreet Boys', 'Bad Religion', 'Balkan Beat Box', 'Bananarama', 'Bastille', 'Ben E King', 'Benjamins Brother', 'Beyoncé', 'Billie Eilish', 'Billy Joel', 'Billy Ray Cyrus', 'Bish Bash', 'Black Eyed Peas', 'Black Lab', 'Blink 182', 'Blue Oyster Cult', 'BoB', 'Bob Dylan', 'Bob Marley', 'Bob Seger', 'Bobby Mcferrin', 'Bobby Vinton', 'Bon Iver', 'Bon Jovi', 'Boyzone', 'Brian Hyland', 'Britney Spears', 'Bruce Springsteen', 'Bruno Mars', 'Bryan Adams', 'Buena Vista Social Club', 'Burhan Çaçan', 'Bülent Serttaş', 'Cake', 'Calvin Harris', 'Camila Cabello', 'Camp Kill Yourself', 'Cansever', 'Carole King', 'Cat Stevens', 'Celine Dion', 'Charlie Puth', 'Chesher', 'Chris Brown', 'Chris Isaak', 'Christina Aguilera', 'Christina Perri', 'Chubby Checker', 'Chuck Berry', 'Clean Bandit', 'Cold Comfort', 'Coldplay', 'Cornershop', 'Counting Crows', 'Cranberries', 'Creed', 'Creedence Clearwater Revival', 'Crowded House', 'Cyndi Lauper', 'DHT', 'DRAKE', 'Daft Punk', 'Damien Rice', 'Daniel Powter', 'Daughtry', 'David Bowie', 'David Gray', 'David Guetta', 'Dean Lewis', 'Deep Purple', 'Del Shannon', 'Dennis Lloyd', 'Deorro', 'Desree', 'Diana Ross', 'Dick Dale', 'Diddy', 'Dido', 'Dire Straits', 'Dirty Fly', 'Disturbed', 'Dolly Parton', 'Dondit John', 'Dr Alban', 'Drake Bell', 'Dschinghis Khan', 'Duffy', 'Duncan Laurence', 'Duran Duran', 'Dusty Springfield', 'Eagle Eye Cherry', 'Eagles', 'Ed Sheeran', 'Eddie Vedder', 'Eddy Grant', 'Edwin Starr', 'Eitan Katz', 'Elkie Brooks', 'Ella Fitzgerald', 'Ellie Goulding', 'Elton John', 'Elvis Costello', 'Elvis Presley', 'Eminem', 'Engin Nurşani', 'Enrique Iglesias', 'Eric Clapton', 'Eurythmics', 'Eva Simons', 'Evanescence', 'Extreme', 'Family of the Year', 'Fatih Reyhan', 'Felix Jaehn', 'Foo Fighters', 'Foreigner', 'Fort Minor', 'Franco Micalizzi', 'Frank Sinatra', 'Frankie Goes To Hollywood', 'Fugees', 'Full Trunk', 'Fun', 'GDFR', 'Gabriel GarzónMontano', 'Garbage', 'Gary Jules', 'Genesis', 'George Benson', 'George Ezra', 'George Michael', 'Geva Alon', 'Gipsy Kings', 'Gloria Gaynor', 'Goo Goo Dolls', 'Gotye', 'Green day', 'Gregory Alan Isakov', 'Guler Duman', 'Guns N Roses', 'Hanson', 'Haris Alexiou', 'Harry Styles', 'Hovi Star', 'Hozier', 'INXS', 'Icona Pop', 'Idina Menzel', 'Imagine Dragons', 'Incubus', 'Israel Kamakawiwoole', 'Jack Johnson', 'Jake Bugg', 'James Arthur', 'James Blunt', 'James Brown', 'James Taylor', 'Jamiroquai', 'Janis Joplin', 'Jason Derulo', 'Jason Mraz', 'Jeff Buckley', 'Jennifer Rush', 'Jess Glynne', 'Jesse Eisenberg', 'Jessica Simpson', 'Jessie Mueller', 'Jewel', 'Jimi Hendrix', 'Joan osborne', 'Jodi Benson', 'Joe Cocker', 'John Denver', 'John Legend', 'John Lennon', 'John Newman', 'John Paul Young', 'Johnny Cash', 'Journey', 'Juanes', 'Judy Garland', 'Justin Bieber', 'Justin Timberlake', 'KT Tunstall', 'KYGO', 'Kaleo', 'Kansas', 'Kanye West', 'Katy Perry', 'Kelly Clarkson', 'Kim Kate Rose', 'Kings Of Leon', 'Kinks', 'Kodaline', 'Kool And The Gang', 'LMFAO', 'Lady Antebellum', 'Lady Gaga', 'Lake  Palmer', 'Lana Del Rey', 'Lauryn Hill', 'Led Zeppelin', 'Lenny kravitz', 'Leo Sayer', 'Leonard Cohen', 'Lewis Capaldi', 'Lifehouse', 'Limp Bizkit', 'Linkin Park', 'Lionel Ritchie', 'Little Richard', 'Lola Marsh', 'Lord Huron', 'Lorde', 'Lost Frequencies', 'Lou Reed', 'Louis Armstrong', 'Lucy Hale', 'Luis Fonsi', 'Lukas Graham', 'Lynyrd Skynyrd', 'MAGIC', 'MIA', 'MOTiD מוטי דרעי', 'MYפסטיגל', 'Madness', 'Madonna', 'Mahmood', 'Major Lazer', 'Manic Street Preachers', 'Maria McKee', 'Mariah Carey', 'Mark Ronson', 'Maroon 5', 'Massive Attack', 'Matisyahu', 'Matthaios Tsahouridis', 'Maya Isacowitz', 'Meghan Trainor', 'Melissa Etheridge', 'Men at Work', 'Meredith Brooks', 'Metallica', 'Michael Jackson', 'Michael Kiwanuka', 'Mika', 'Mike Brant', 'Mike Oldfield', 'Milky chance', 'Moby', 'Mona Lupa', 'Monsters and Men', 'Monty Python', 'Morland', 'Mr Big', 'Mr Probz', 'Muazzez Ersoy', 'Muse', 'Måns Zelmerlöw', 'NSYNC', 'Nadav Guedj', 'Najwa Farouk', 'Natalie Imbruglia', 'Neil Diamond', 'Neil Young', 'Nelly Furtado', 'Netta Barzilai', 'New Order', 'Nickelback', 'Nicky Jam', 'Nico  Vinz', 'Nikos Vertis', 'Nina Simone', 'Nirvana', 'No Doubt', 'Noah and the Whale', 'Noga Erez', 'Nylons', 'OMI', 'Oakey Philip', 'Oasis', 'Omer Netzer', 'One Direction', 'OneRepublic', 'Ori Ronen', 'Ori Ronen  The Folk Stoners', 'Panic At the disco', 'Passenger', 'Paul Simon', 'Pearl Jam', 'Peter Bjorn and John', 'Pharrell Williams', 'Phil Collins', 'Phoebe Buffet', 'Pink', 'Pink floyd', 'Pitbull', 'Pixies', 'Placebo', 'Plain White Ts', 'Porcupine Tree', 'Powfu', 'Prince', 'Queen', 'REM', 'Radiohead', 'RagnBone Man', 'Ralph McTell', 'Randy Newman', 'Ray Charles', 'Reamomn', 'Red Hot Chili Peppers', 'Reggae Shark', 'Ricardo Silva', 'Rick and Morty', 'Ricky Martin', 'Riff Cohen', 'Rihanna', 'Robbie Williams', 'Rockwell', 'Rod Stewart', 'Rolling Stones', 'Roxette', 'Roxy Music', 'Roy Orbison', 'Salt N Pepa', 'Sam Cooke', 'Sam Smith', 'Santana', 'Scarlett Johansson', 'Scorpions', 'Sean Paul', 'Seccad Mehmedi', 'Selfish Shepherd', 'Shakira', 'Shannon Purser', 'Shawn Mendes', 'Sheryl Crow', 'Shocking Blue', 'Shontelle', 'Sia', 'Simon  Garfunkel', 'Simple Plan', 'Sivan Talmor', 'Sixpence None the Richer', 'Skeeter Davis', 'Slipknot', 'Smash Mouth', 'Smokie', 'Snap', 'Snow Patrol', 'Snow White', 'Soggy Bottom Boys', 'Sokratis Malamas', 'Spandau Ballet', 'Spice Girls', 'Spin Doctors', 'Staind', 'Status Quo', 'Stavros Pazarentsis', 'Steppenwolf', 'Stereo MCs', 'Steve Miller Band', 'Stevie Wonder', 'Sting', 'Styx', 'Sugabebes', 'Survivor', 'Susanne Sundfør', 'Suzanne Vega', 'Swan Band  סוואן בנד', 'System Of A Down', 'Take That', 'Talking Heads', 'Tasmin Archer', 'Taylor Swift', 'Tears for Fears', 'Tenacious D', 'The Angelcy', 'The Animals', 'The Beach Boys', 'The Beatles', 'The Buggles', 'The Chainsmokers', 'The Clash', 'The Commitments', 'The Communards', 'The Corrs', 'The Cure', 'The Doors', 'The Everly Brothers', 'The Fray', 'The Lumineers', 'The Moody Blues', 'The Parakit', 'The Paz Band', 'The Police', 'The Presidents of the United States of America', 'The Rembrandts', 'The Righteous Brothers', 'The Script', 'The Secret Sea', 'The Shadows', 'The Smashing Pumpkins', 'The Tokens', 'The Turtles', 'The Verve', 'The Wannadies', 'The Weeknd', 'The Zombies', 'Tim Maia', 'Tom Jones', 'Tom Petty', 'Tones And I', 'Tori Kelly', 'Tracy Chapman', 'Train', 'Travis', 'Twenyy One Pilots', 'U2', 'Urge Overkill', 'Useless ID', 'Uğur Karakuş', 'Van Morrison', 'Vance Joy', 'Vanessa Carlton', 'Vitas', 'WIZ KHALIFA', 'Wheatus', 'Years Around The Sun', 'Yitzi Hurwitz', 'Yves Larock', 'ZAYN', 'Zara Larsson', 'Ziv Zaifman', 'Zusha', 'cream', 'iogi', 'skunk Anansie', 'whitney houston', 'Μανώλης Μπαρμπεράκης', 'אבטיפוס', 'אבי אבורומי', 'אבי אילסון', 'אבי ביטר', 'אבי בן ישראל', 'אבי בניון', 'אבי גרייניק', 'אבי טולדנו', 'אבי מסיקה', 'אבי פיאמנטה', 'אביאור מלסה', 'אביב אלוש', 'אביב בכר', 'אביב גדג', 'אביב גפן', 'אביגיל רוז', 'אביה מלכה', 'אביהו מדינה', 'אביהו פנחסוב', 'אביהו שבת', 'אביחי הולנדר', 'אבירם צור', 'אבישי כהן', 'אביתר בנאי', 'אבן שוק', 'אבנר גדסי', 'אברהם טל', 'אברהם פריד', 'אברהם שלונסקי', 'אברומי פלאם', 'אגם בוחבוט', 'אגם רודברג', 'אדיר גץ', 'אדיר והילדות', 'אדם', 'אדם בן אמיתי', 'אדם מילשטיין', 'אדם סול', 'אדר גולד', 'אהובה עוזרי', 'אהוד בנאי', 'אהוד מנור', 'אהרון רזאל', 'אודי דוידי', 'אודי כגן', 'אודיה', 'אוהד חיטמן', 'אוהד שרגאי', 'אולי דנון', 'אוף שימחעס', 'אופיר אלחייני', 'אופל רביבו', 'אור אדרי', 'אור בן ברוך', 'אור בר', 'אור עמרמי ברוקמן', 'אוראל גיספן', 'אורות', 'אורי בן ארי', 'אורי בנאי', 'אורי הרפז', 'אורי ויזל', 'אורי מזרחי', 'אורי פיינמן', 'אורי שכיב', 'אוריאל מאיר', 'אוריאל שלומי', 'אוריאן שטאל', 'אורית מרק אטינגר', 'אורן ברזילי', 'אורן חן', 'אורן לוטנברג', 'אורן צור', 'אורן שבו', 'אורנה דץ', 'אושיק לוי', 'אושר ביטון', 'אושר כהן', 'אזולאי נמט', 'אחי נתן', 'אחינועם ניני', 'אחרית הימים', 'אטרף', 'איב אנד ליר', 'איבי הנחל', 'איברהים טטליסס', 'איגי וקסמן', 'איה זהבי פייגלין', 'איה כורם', 'איזבו', 'איזי', 'אייל אבן צור', 'אייל גולן', 'אייל דביר', 'אייל ישראל', 'אייל קומרובסקי', 'איילה סיני', 'איל כהן', 'אילן נורי', 'אילנה אביטל', 'אילנה רובינא', 'אילנית', 'אינפקציה', 'איפה הילד', 'איציק דדיה', 'איציק פצצתי', 'איציק קדם', 'איציק קלה', 'איציק ריקן', 'איציק שמלי', 'אישתאר', 'איתי גורן', 'איתי לוי', 'איתי פרל', 'איתן גינצבורג', 'איתן דרמון', 'איתן מסורי', 'איתן פלד', 'אלגיר', 'אלדד כהן', 'אלדד ציטרין', 'אלהלי להב', 'אלון אולארציק', 'אלון עדר', 'אלון עדר ולהקה', 'אלחנן מן הבאר', 'אלי לוזון', 'אלי לולאי', 'אלי מגן', 'אלי פרידמן', 'אליאנה תדהר', 'אליה גבאי', 'אליה רוסיליו', 'אליעד נחום', 'אליעזר בוצר', 'אליקום תור', 'אליקם בוטה', 'אלירז זאדה', 'אלירן אסלן', 'אלישע בירנבאום', 'אלישע בנאי וארבעים השודדים', 'אלכסנדר אילאי אשרי', 'אלמוג קריאף', 'אלני ויטאלי', 'אלעד שער', 'אלרואי', 'אמיר דדון', 'אמיר ובן', 'אמיר חצרוני', 'אמיר פיי גוטמן', 'אמיתי סאסי', 'אמן', 'אמני ישראל', 'אנדד שני', 'אנדרדוס', 'אנטוניס ורדיס', 'אניה בוקשטיין', 'אנסמבל יעלה', 'אסטמה ושקד פררה', 'אסנת פז', 'אסנת רביב', 'אסף אברבוך והשמועה', 'אסף אור', 'אסף אמדורסקי', 'אסף גורן', 'אסף גרנות', 'אסתר שמיר', 'אפור גשום', 'אפק לאמור', 'אפרים שמיר', 'אפרת בוימולד', 'אפרת גוש', 'ארד ריכטר', 'ארז הלוי', 'ארז לב ארי', 'ארז לבנון היד', 'ארז לונברג', 'ארי גורלי ורעות יהודאי', 'אריאל הורוביץ', 'אריאל זילבר', 'אריאל עובדיה', 'אריק איינשטיין', 'אריק ברמן', 'אריק לביא', 'אריק משעלי', 'אריק סיני', 'ארמדילוס', 'ארץ נהדרת', 'ארקדי דוכין', 'אתי אנקרי', 'אתי לוי', 'אתי קסטרו', 'אתל', 'אתניקס', 'בובי רזולו', 'בובספוג', 'בוטן מתוק בקרקס', 'בועז בנאי', 'בועז מעודה', 'בועז עופרי', 'בטי פבלו', 'ביטניק', 'בית הבובות', 'בלאגן', 'בן אדרי', 'בן אל', 'בן אל תבורי', 'בן ארצי', 'בן גולן', 'בן גורן', 'בן לולו', 'בן סנוף', 'בנזין', 'בני אלבז', 'בני אמדורסקי', 'בני ברמן', 'בני בשן', 'בני פרידמן', 'בניה ברבי', 'בנימין לנדאו', 'בנצי צור', 'בנשר', 'בעז שרעבי', 'בציר 76', 'בר גרוצקי', 'בר מזוז', 'בר צברי', 'ברוך לוין', 'ברי סחרוף', 'בתאל בןזקן', 'גאולה ציטרינוביץ', 'גאיה', 'גאליס', 'גבי שושן', 'גבריאל בלחסן', 'גד אלבז', 'גדי פיינגולד', 'גדעון אונה', 'גוזי כץ', 'גוטה גוטה', 'גולי והגיטרה ששרה', 'גולן אטיאס', 'גוני שועלי', 'גורי אלפי', 'גזוז', 'גיא זוארץ', 'גיא לוי', 'גיא מזיג', 'גיא מנטש', 'גיא שמי', 'גידי גוב', 'גידי דבוש', 'גיים בויז', 'גיין בורדו', 'גיל בן דוד', 'גיל בר הדס', 'גיל ויין', 'גיל עקיביוב', 'גילה אדרי', 'גילי ארגוב', 'גילי וגלית', 'גימבו גיי', 'גינגיות', 'גירפות', 'גל אדם', 'גל חייק', 'גל טבת', 'גל מלכה', 'גל שריג', 'גלי עטרי', 'גליקריה', 'גלית בל', 'גלעד ויטל', 'גלעד כהנא', 'גלעד פוטולסקי', 'גלעד פרי', 'גלעד שגב', 'גן חיות', 'גני תמיר', 'גקו אייזנברג', 'גקי אליהו', 'גקי מקייטן', 'גקי שושן', 'גרי אקשטיין', 'דאנה איבגי', 'דב שורין', 'דבק', 'דבקות', 'דדי דדון אסי כהן מארץ נהדרת', 'דוד בודה', 'דוד בן ארזה', 'דוד דאור', 'דוד חתוכה', 'דוד לביא', 'דוד ליפשיץ', 'דוד מעיין', 'דודה', 'דודו אהרון', 'דודו דרעי', 'דודו טסה', 'דודו קדוש', 'דודו רז ביטון', 'דודי בוזגלו', 'דודי בר דוד', 'דודי לוי', 'דולי ופן', 'דור דניאל', 'דורי בן זאב', 'דורית ראובני', 'דורשי ה', 'דיאנה גולבי', 'דיג דיג דוג', 'דיגיי אונלי וטליסמאן', 'דידי ארז', 'דיויד ברוזה', 'דין דין אביב', 'דין כרמל', 'דיסני', 'דיקלה', 'דן פית  תנועת תרבות', 'דן תורן', 'דנה אינטרנשיונל', 'דנה ברגר', 'דנה דבורין', 'דנה לב', 'דנה לפידות', 'דנה עדיני', 'דני בסן', 'דני ליטני', 'דני סנדרסון', 'דני רובס', 'דניאל ברזילאי', 'דניאל דרחי', 'דניאל זילברשטיין', 'דניאל זמיר', 'דניאל סלומון', 'דניאל סמירה', 'דניאל רובין', 'דניאל תבורי', 'דניאלה חגאי', 'דניאלה ספקטור', 'דנידין', 'דפנה ארמוני', 'דפנה דקל', 'דקל וקנין', 'דקלון', 'דתיה בן דור', 'האולטראס', 'האחיות גמשיד', 'האחיות יוספי', 'האחיות כרקוקלי', 'האחים בן עזרא', 'האחים והאחיות', 'האחים צברי', 'הבורגנים', 'הבטלנים', 'הבילויים', 'הבליינים', 'הבנות נחמה', 'הברירה הטבעית', 'הגבעטרון', 'הגר יפת', 'הגשש החיוור', 'הדג נחש', 'הדודאים', 'הדורבנים', 'הדס קליינמן', 'הופה היי', 'הזבובים', 'החברים משער יפו', 'החברים של נטאשה', 'החדשות', 'החלונות הגבוהים', 'החצר האחורית', 'הטוב הרע והנערה', 'הטרמפולינה', 'היהודים', 'היוצרים', 'היי פייב', 'הילה הררי', 'הינדיק', 'הכבש השישה עשר', 'הכל עובר חביבי', 'הלהקה', 'הלל פלאי', 'הלם תרבות', 'המדרגות', 'המופע של ויקטור גקסון', 'המורנוס', 'המושבה', 'המין האנושי', 'המכשפות', 'המפשעות', 'המשולש', 'הנשמות הטהורות', 'העברית', 'הפיגמות', 'הפיל הכחול', 'הפסגות', 'הפרברים', 'הפרוייקט של עידן רייכל', 'הפרויקט של רביבו', 'הפרלמנט', 'הצאקרה השמינית', 'הצל', 'הקומדי סטור', 'הראל טל', 'הראל מויאל', 'הראל סקעת', 'הרב רונן חזיזה', 'הרב רפאל רובין', 'הרפו והנשמות', 'הרצל קביליו', 'השובלים צלילי שבזי', 'השוונג של הפיתה', 'השלושרים', 'השמחות', 'התבלינים', 'התקווה 6', 'התרנגולים', 'ויק אוחנה', 'ורד סיסם שחר', 'זאב טנא', 'זאק פקיאל מתוך פלאשבק', 'זבל', 'זהבה בן', 'זוהר ארגוב', 'זוהר אשירוב', 'זוהר חיון', 'זוהרית ממן', 'זפיריס מילאס', 'זקני צפת', 'חבד', 'חבורת רננים', 'חובבי ציון', 'חוה אלברשטיין', 'חיילי הנקמה', 'חיים איפרגן', 'חיים גולד', 'חיים ישראל', 'חיים משה', 'חיים צינוביץ', 'חיל האוויר', 'חיל הים', 'חיל חינוך', 'חיליק פרנק', 'חלב ודבש', 'חם ומחמם בום', 'חמאס', 'חמי רודנר', 'חן אהרוני', 'חן בן אליהו', 'חן גז', 'חן יעקובסון', 'חן ליבנדמן', 'חן פורתי', 'חני נחמיאס', 'חנן בן ארי', 'חנן יובל', 'חנן ילד הפלא', 'חננאל', 'חסידות מודזיץ', 'טוביה צפיר', 'טוהר', 'טום קשתי', 'טונה', 'טיפקס', 'טל בלרכוביץ', 'טל ורסנו', 'טל טירנגל', 'טל ירימי וט3', 'טל מדר', 'טל מנחם', 'טל סונדק', 'טל רמון', 'טל שגב', 'טליסמאן', 'יאיר אלייצור', 'יאיר גדסי', 'יאיר לוי', 'יאיר מגיד', 'יגאל בשן', 'יגאל כרמל', 'יגל', 'יהודה אליאס', 'יהודה דים', 'יהודה סעדו', 'יהודה פוליקר', 'יהודית רביץ', 'יהודית תמיר', 'יהונתן אבידני', 'יהונתן גפן', 'יהורם גאון', 'יהושע לימוני', 'יהלי סובול', 'יואב גל', 'יואב יצחק', 'יואל שמש', 'יואלי קליין', 'יובל בנאי', 'יובל דה לוי', 'יובל דיין', 'יובל המנוול', 'יובל טייב', 'יובל מנדלסון', 'יובל סלע', 'יובל שםטוב יובל המבולבל', 'יוגב גלוסמן', 'יוחאי מורנו', 'יונה קפלן', 'יוני אסף', 'יוני בלוך', 'יוני גנוט', 'יוני גרייב', 'יוני רועה', 'יוני רכטר', 'יונינה', 'יונתן ביטון', 'יונתן כהן', 'יונתן רוזן', 'יונתן רזאל', 'יונתן שינפלד', 'יוסי אזולאי', 'יוסי אלפנט', 'יוסי בנאי', 'יוסי גבני', 'יוסי וטירן', 'יוסי עדן', 'יוסי עסיס', 'יוסי שטרית', 'יוסף חיים שוואקי', 'יוסף נטיב', 'יוסף קרדונר', 'יורגוס דאלארס', 'יוריק בן דוד', 'יורם טהרלב', 'יורם פתל', 'יושי', 'יותם בן חורין', 'יזהר אשדות', 'יזהר כהן', 'ילדות רעות', 'ילדי הפסטיגל', 'ים רפאלי', 'יניב בן משיח', 'יניב בר', 'יסמין מועלם', 'יעל לוי', 'יעקב חתן', 'יעקב שוואקי', 'יפה ירקוני', 'יפתי', 'יצחק מאיר', 'יצחק קלפטר', 'יציאת חירום', 'יקיר ונה', 'ירדנה ארזי', 'ירון חדד', 'ירמי קפלן', 'ישי לוי', 'ישי רביבו', 'ישי ריבו', 'ישיבת רמת גן', 'ישראל בר און', 'ישראל דגן', 'ישראל פרנס', 'ישראל קלר', 'כוורת', 'כוכב אניס', 'כוכבי השמינייה', 'כל החתיכים אצלי', 'כלא 6', 'כנסיית השכל', 'כפיר אפשטיין', 'כרמי זיסאפל', 'כרמלה גרוס ואגנר', 'כשניקו תתחיל לדבר', 'לא ידוע', 'לאה שבת', 'להבה', 'להקת stardust', 'להקת גייסות השריון', 'להקת הנוער העובד והלומד', 'להקת הנחל', 'להקת חופים', 'להקת חינוך מיוחד', 'להקת פיקוד מרכז', 'להקת פיקוד צפון', 'להקת קולות', 'להקת רקיע', 'להקת שיר חדש', 'להקת שלוה', 'לוי קמינשטיין', 'לולה', 'לולה מארש', 'לורי', 'לורן פלד', 'לי בירן', 'ליאור אלמליח', 'ליאור יוסף שביב', 'ליאור ייני', 'ליאור נרקיס', 'ליאור פרחי', 'ליאורה', 'ליאת אחירון', 'ליאת אליהו', 'ליאת יצחקי', 'לידור ויצמן', 'לידור יוספי', 'לידור סולטן', 'ליהי בנין', 'ליילי', 'לימור עובד', 'לינט', 'ליפא שמלצר', 'לירון עמרם', 'לירון רמתי', 'לירן דנינו', 'לירן נדל', 'מאור אדרי', 'מאור אשואל', 'מאור גמליאל', 'מאור כהן', 'מאור תיתון', 'מאיה אברהם', 'מאיה בוסקילה', 'מאיה הרמן', 'מאיה רוטמן', 'מאיר אלפי', 'מאיר אריאל', 'מאיר בנאי', 'מאיר כהן', 'מארינה מקסימיליאן', 'מוטי ביקובסקי', 'מוטי וייס', 'מוטי טקה', 'מוטי מונק', 'מוטי פליישר', 'מוטי שטיינמץ', 'מומי לוי', 'מונטי פיורי', 'מוניקה סקס', 'מונסודיום', 'מופע הארנבות של דר קספר', 'מוקי', 'מור', 'מור מזרחי', 'מורין נהדר', 'מורן אהרוני', 'מורן גמליאל', 'מורן לוביץ', 'מורן מזור', 'מורן קריטי', 'מוש בן ארי', 'מושיק עפיה', 'מושיקו ינגילוב', 'מושיקו מור', 'מזי כהן', 'מחסון קירמיזיגול', 'מטרופולין', 'מידד טסה', 'מיה גרידיש', 'מיה יוהנה', 'מיטל טרבלסי', 'מיטל מיכאלי', 'מיי פיינגולד', 'מיילך קאהן', 'מיכאל גריילסאמר', 'מיכאל לוי', 'מיכאל שטרייכר', 'מיכה ביטון', 'מיכה שטרית', 'מיכל אמדורסקי', 'מיכל הקטנה', 'מיכל נחמן', 'מיכל שפירא', 'מילי מירן', 'מיקה הרי', 'מיקה קרני', 'מיקי גבע', 'מיקי גבריאלוב', 'מיקי קם', 'מירי אלוני', 'מירי מסיקה', 'מישל', 'מכלוף', 'מלך האריות', 'מלכה באיה', 'מנדי הרשקוביץ', 'מנור שבת', 'מנחי ערוץ הילדים', 'מני בגר', 'מני ממטרה', 'מני עזרא', 'מניפסט', 'מעיין', 'מעלה  המרכז לציונות דתית', 'מפרשים', 'מצדה איש ימיני', 'מקהלת הרבנות הצבאית', 'מקהלת חסידי יאסי', 'מקלט 31', 'מרב סימן טוב', 'מרגי', 'מרגלית צנעני', 'מרדכי בן דוד', 'מרדכי ברוצקי', 'מרדכי שפירא', 'מרטין דוידסון', 'מרסדס בנד', 'משה בקר', 'משה גולדמן', 'משה גיאת', 'משה דץ', 'משה לוי', 'משה נריה כורסיה', 'משה פלד', 'משה פרץ', 'משי קלינשטיין', 'משיח', 'משינה', 'משפחת גוטהלף', 'משפחת ואך', 'מתי בורשטין', 'מתי כספי', 'מתי שריקי', 'מתן גלילוב', 'נאוה תהילה', 'נאור אורמיה', 'נאור כהן', 'נדב יעיש', 'נוי פדלון', 'נועה קירל', 'נועם איתן', 'נועם בנאי', 'נועם דדון', 'נועם מיכאל', 'נועם נתיב', 'נועם עמבר', 'נועם קניאל', 'נועם רותם', 'נוער שוליים', 'נופר סלמאן', 'נורי קורן', 'נורית גלרון', 'נושאי המגבעת', 'נחזיר תחונזיר', 'נחמה הנדל', 'נטלי פרץ', 'ניב גולד', 'ניב דמירל', 'ניב מנצור', 'נילי פינק', 'נינט טייב', 'ניסו סימן טוב', 'ניסים גרמה', 'ניסים סרוסי', 'ניקה', 'ניר מנשה', 'ניר פרידמן', 'נירה גל', 'נירה רבינוביץ', 'ניתן בן ארי', 'נמרוד והבודדים', 'נמרוד לב', 'נמרוד שיקלר', 'נסרין קדרי', 'נעליים', 'נעם אמירא', 'נעמי שמר', 'נערות ריינס', 'נערי רפול', 'נפתלי הרפז', 'נפתלי הרץ אימבר', 'נפתלי קמפה', 'נצי נצ רביד פלוטניק', 'נקמת הטרקטור', 'נרקיס', 'נתי לוי', 'נתן גושן', 'נתנאל', 'נתנאל גולדברג', 'נתנאל פרי', 'נתנאל ששון', 'נתנאלה', 'סאבלימינל', 'סגול', 'סגול 59', 'סגיב כהן', 'סהר שרב', 'סטטיק', 'סטלה מאריס', 'סטליוס קזנגידיס', 'סטמטיס גונידיס  stamatis gonidis', 'סי היימן', 'סיון שביט', 'סיני תור', 'סינרגיה', 'סמיר שוקרי', 'סער בדישי', 'ספיר מתתיהו', 'ספיר סבן', 'סקסטה', 'סשה גרישקוב', 'עבודות עפר', 'עברי לידר', 'עדי ביטי', 'עדי דואניס', 'עדי לוי', 'עדי רן', 'עדיאל טיירי', 'עדן', 'עדן אלנה', 'עדן בן זקן', 'עדן גבאי', 'עדן חסון', 'עדן מאירי', 'עדן שאבי', 'עדנה לב', 'עובדיה חממה', 'עוזי חיטמן', 'עוזי נבון', 'עוזי פוקס', 'עוזיה צדוק', 'עומר אדם', 'עומר שמחוני', 'עומרי סבח', 'עופר אקרלינג', 'עופר בדיחי', 'עופר לוי', 'עזרא גטנו', 'עטר מיינר', 'עידו אנגלר', 'עידו ונה', 'עידית אשל', 'עידן בקשי', 'עידן חביב', 'עידן יניב', 'עידן עמדי', 'עילי בוטנר', 'עינב גקסון כהן', 'עינת שמל', 'עירית סנדנר', 'עלובי החיים', 'עלי מוהר', 'עליית אוקטובר', 'עלמא', 'עלמה זוהר', 'עלמה קלברמן', 'עמי אבו', 'עמי נגר', 'עמיר בניון', 'עמיר לב', 'עמית אולמן פדרו גראס', 'עמית פרקש', 'עממי', 'ענבל ביבי', 'ענבל פרלמוטר', 'ענת מלמוד', 'ענת עצמון', 'עספור', 'עפר בשן', 'עפרה חזה', 'עקיבא תורגמן', 'ערופי שפתיים', 'ערן וקנין', 'ערן צור', 'ערפדי הרגטיים', 'פאנקייק', 'פאנקנשטיין', 'פאר טסי', 'פבלו רוזנברג', 'פוטו מריאנה', 'פוציםפוצים', 'פורטרט', 'פורת נוב', 'פטריק סבג', 'פיטר רוט', 'פינג פונג', 'פינה בעמק', 'פיני איינהורן', 'פיני חדד', 'פינקי', 'פיקוד דרום', 'פרחי טורונטו', 'פרחי מיאמי', 'פרישמן והחלוצים', 'פשוטי העם', 'צביקה פיק', 'צוות הווי של הנחל', 'צח חכים', 'צחי קליין', 'צילה דגן', 'ציפי שביט', 'צליל דיין', 'צליל מכוון', 'צמד דרום', 'צמד העמרנים', 'קובי אוז', 'קובי אפללו', 'קובי אשרת', 'קובי בוקעי', 'קובי ברומר', 'קובי ממן', 'קובי מרימי', 'קובי פרץ', 'קוקו', 'קוקי לבנה', 'קורין אלאל', 'קורל חצבני', 'קטריקס', 'קפה שחור חזק', 'קצר', 'קרולינה', 'קרח 9', 'קריזמו', 'קרן פלס', 'קרן קריספיל', 'ראובן גבירץ', 'רבי שמואל ברזיל', 'רבי שניאור זלמן מלאדי', 'רביב כנר', 'רבע לאפריקה', 'רבע לשבע', 'רבקה זוהר', 'רבקה מיכאלי', 'רגב הוד', 'רובי לוי', 'רוחמה רז', 'רון בוחניק', 'רון ביתן', 'רון כהן', 'רון מרגן', 'רון נשר', 'רון שובל', 'רונה קינן', 'רוני אחיאל', 'רוני אלטר', 'רוני דואני', 'רוני דלומי', 'רוניוס', 'רונית רולנד', 'רונית שחר', 'רונן שטרן', 'רועי דהן', 'רועי ידיד', 'רועי כפרי', 'רועי מולה', 'רועי נויפלד', 'רועי סנדלר', 'רועי שרעבי', 'רוקפור', 'רות גונזלס', 'רותי נבון', 'רותם כהן', 'רז חדד', 'רז שמואלי', 'רחל ירון', 'ריטה', 'רינת בר', 'רינת גבאי', 'ריקי גל', 'רמי פורטיס', 'רמי קלינשטיין', 'רמלה', 'רן אלירן', 'רן דנקר', 'רן שפיר', 'רן שריג', 'רפי פרסקי', 'שאזאמאט', 'שאנן סטריט', 'שבי בר', 'שבע', 'שבק ס', 'שגב הרוש', 'שגיא ברייטנר', 'שגיא סרי', 'שוהם טפיארו', 'שוטי הנבואה', 'שולה חן', 'שולי נתן', 'שולי רנד', 'שונות', 'שוקולד מנטה מסטיק', 'שוקי ודורית', 'שושנה דמארי', 'שחר אבן צור', 'שחר אדמוני', 'שחר ברניצן', 'שי אברמסון', 'שי גבסו', 'שי המבר', 'שי עמר', 'שי צברי', 'שי שרעבי', 'שיבי קלר', 'שייגעצ', 'שייקה לוי', 'שילה פרבר', 'שילי עטרי', 'שימי מזרחי', 'שימי תבורי', 'שיר זוארץ', 'שיר לוי', 'שיר קורן', 'שירה גבריאלוב', 'שירה לוי', 'שירה מרגלית', 'שירי מימון', 'שירי מרפסת רועי כפרי וטל טירנגל', 'שירלי צפרי', 'שירן בן טוב', 'שלהבת', 'שלו אדרי', 'שלום גד', 'שלום חנוך', 'שלומי ברכה', 'שלומי דסקל', 'שלומי סרנגה', 'שלומי שבן', 'שלומי שבת', 'שלי שרון', 'שלישיית גשר הירקון', 'שלישיית מה קשור', 'שלישיית קצה השדה', 'שלישיית שריד', 'שלמה ארצי', 'שלמה בר', 'שלמה גרוניך', 'שלמה יידוב', 'שלמה כהן', 'שלמה קרליבך', 'שם טוב האבי', 'שם טוב לוי', 'שמוליק קראוס', 'שממל', 'שמעון בוסקילה', 'שמעון גבלץ', 'שני יצהרי', 'שני פונה', 'שפי אורן בך', 'שפיות זמנית', 'שפתיים', 'שקד קוממי', 'שקל', 'שר אל', 'שרוטונים', 'שרון זריהן', 'שרון חזיז', 'שרון ליפשיץ', 'שרון קוניץ', 'שרון רוטר', 'שרונה ודניאלה פיק', 'שרי', 'שרי צוריאל', 'שריף', 'שרית אביטן', 'שרית חדד', 'ששי קשת', 'ששת', 'תום אבני', 'תום אקו', 'תומר חן', 'תומר יוסף', 'תומר ישעיהו', 'תומר לוי', 'תיסלם', 'תיקי דיין', 'תירס סקסואל', 'תמוז', 'תמי פדרמן', 'תמיר גל', 'תמיר סולטנה', 'תמר גלעדי', 'תערובת אסקוט'] {allow-input: true}

{type:"raw"}
def artist_songs(artist_name=str):
  #song_list = df.loc[df['אמן']==artist_name]['שיר'].sort_values()
  song_list = df.loc[df['אמן'].str.contains(artist_name)]['שיר'].sort_values()

  song_list = list(zip(song_list,song_list.index))
  return song_list
  
song_tab = widgets.Select(options=artist_songs(Artist),
                          description='Song:',
                          rows=20)
display(song_tab)

Select(description='Song:', options=(('(You Make Me Feel Like) A Natural Woman', 4484), ('10 שניות', 1774), ('…

In [ ]:
#Artist = 
temp_list = artist_list
print(temp_list)

['3 Doors Down', '4 Non Blondes', '5 Seconds of Summer', '8th Day', 'A1', 'ABBA', 'ACDC', 'ANNA RF', 'Acollective', 'Adam Ant', 'Adam Levine', 'Adele', 'Adi Ulmansky', 'Aerosmith', 'Afroman', 'Aha', 'Air', 'Air Supply', 'Airbag', 'Al Green', 'Aladdin', 'Alan Parsons Project', 'Alan Walker', 'Alanis Morissette', 'Alec Benjamin', 'Alekos Zazopoulos', 'Alex Turner', 'Alfred García', 'Alice In Chains', 'Alicia keys', 'All Saints', 'Aloe Blacc', 'Alphaville', 'America', 'Amy Macdonald', 'Amy Winehouse', 'Anouk', 'Arctic Monkeys', 'Aretha Franklin', 'Arif Ceylan', 'Aris San', 'Arrhwallaco', 'Asaf Avidan', 'Asaf Avidan  the Mojos', 'Augustana', 'Avenged Sevenfold', 'Avicii', 'Avril Lavigne', 'Awolnation', 'Backstreet Boys', 'Bad Religion', 'Balkan Beat Box', 'Bananarama', 'Bastille', 'Ben E King', 'Benjamins Brother', 'Beyoncé', 'Billie Eilish', 'Billy Joel', 'Billy Ray Cyrus', 'Bish Bash', 'Black Eyed Peas', 'Black Lab', 'Blink 182', 'Blue Oyster Cult', 'BoB', 'Bob Dylan', 'Bob Marley', 'Bob

In [ ]:
#@title **Copy the URL** to the desired song from www.tab4u.co.il { run: "auto" }

from IPython.display import Markdown as md
fr=2 #GHz
md("$f_r = %i$ GHz"%(fr))

URL =  "https:" #@param {type:"string"}
md("Progression_value = "0" #@param [6,5,4,3,2,1,0,-1,-2,-3,-4,5])

In [ ]:
def artist_songs(artist_name=str):
  song_list = df.loc[df['אמן']==artist_name]['שיר'].sort_values()
  song_list = list(zip(song_list,song_list.index))
  return song_list
artist_songs(inputs.children[0].value)

In [ ]:
rows_num = 20
artist_tab = widgets.Select(options=artist_list,
                            description='Artist:',
                            rows=rows_num)
rows_num = 20
def artist_songs(artist_name=str):
  song_list = df.loc[df['אמן']==artist_name]['שיר'].sort_values()
  song_list = list(zip(song_list,song_list.index))
  return song_list
  
song_tab = widgets.Select(options=artist_songs(artist_tab.value),
                          description='Song:',
                          rows=rows_num)


display(artist_tab,song_tab)

In [ ]:
rows_num = 20
inputs = widgets.HBox([widgets.Select(
                          options=artist_list,
                          value='ברי סחרוף',
                          description='Artist:',
                          rows=rows_num),
                      widgets.Select(
                          options=song_list,
                          description='Song:',
                          value=545,
                          rows=rows_num)
                      ])

def artist_songs(artist_name=str):
  song_list = df.loc[df['אמן']==artist_name]['שיר'].sort_values()
  song_list = list(zip(song_list,song_list.index))
  return song_list
artist_songs(inputs.children[0].value)
def update_x_range(*args):
    x_widget.options = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

display(inputs)

In [ ]:
from ipywidgets import Layout,jslink

button = widgets.Button(description='Click here')
slider = widgets.IntSlider()
stacked = widgets.stacked([button, slider])
dropdown = widgets.Dropdown(options=['button', 'slider'])
widgets.jslink((dropdown, 'index'), (stacked, 'selected_index'))
widgets.VBox([dropdown, stacked])

In [ ]:
print(artist.value,artist.label)

In [ ]:
df.iloc[545]

אמן                                             ברי סחרוף
שיר                                       עוד חוזר הניגון
לינק    https://www.tab4u.com/tabs/songs/940_%D7%91%D7...
Name: 545, dtype: object

In [ ]:
from ipywidgets import Layout,Stacked

artist = widgets.Select(
          options=artist_list,
          value='ברי סחרוף',
          rows=10,
          description='Artist:',
          layout=Layout(width='20%', height='400px',right='True'),
          disabled=False)
display(artist)

In [ ]:
artist.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_options_labels',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'description',
 'description_tooltip',
 'disabled',
 'index',
 'layout',
 'rows',
 'style']

In [ ]:
artist.style.keys

['_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'description_width']

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

for i in range(200):
  print(i)

In [ ]:
#!pip install ipywidgets
import ipywidgets as widgets
#from IPython.display import display


In [ ]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='',
    description='Artist:',
    disabled=False,
    layout={'width': 'max-content'}
)

#display(output_year)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=['A Scores', 'B Scores']),
                 cells=dict(values=[[100, 90, 80, 90], [95, 85, 75, 95]]))
                     ])
fig.show()
